In [1]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
import os
import cv2
import csv
import time 

In [2]:
model = YOLO("best.pt")

In [3]:
microsleep_counter = 0
yawning_counter = 0

In [4]:
field_names= ['preprocess', 'inference', 'postprocess']
latency = []

In [5]:
cap = cv2.VideoCapture(0)

last_detected_eye = datetime.now()
last_detected_yawn = datetime.now()

# Time-based thresholds in seconds
MICROSLEEP_DURATION = 0.5
YAWNING_DURATION = 3

prev_frame_time = 0
new_frame_time = 0

fps = 30  # Initial guess for FPS, updated dynamically
last_frame_time = time.time()

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()

    # Make detections 
    results = model(frame)

    # Calculate FPS dynamically
    current_time = time.time()
    frame_latency = current_time - last_frame_time  # Time for processing one frame
    last_frame_time = current_time
    fps = 1 / frame_latency  # FPS is the inverse of frame latency

    # Dynamically calculate frame thresholds
    microsleep_thresh = int(MICROSLEEP_DURATION * fps)
    yawning_thresh = int(YAWNING_DURATION * fps)

    boxes = results[0].cpu().numpy().boxes
    detected_class = None

    for box in boxes:
        cls = int(box.cls[0])
        conf = box.conf[0]
    
        if conf > 0.5:
            detected_class = cls
            break
            
    if detected_class == 0:  # Microsleep
        microsleep_counter += 1
        yawning_counter = 0  # Reset yawning counter
    elif detected_class == 2:  # Yawning
        yawning_counter += 1
        microsleep_counter = 0  # Reset microsleep counter
    else:  # Neutral
        microsleep_counter = 0
        yawning_counter = 0

    # Trigger alerts if thresholds are exceeded
    if microsleep_counter >= microsleep_thresh:
        last_detected_eye = datetime.now()
        cv2.putText(frame, 'ALERT: Microsleep', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    elif (datetime.now() - last_detected_eye).total_seconds() < 2:
        cv2.putText(frame, 'ALERT: Microsleep', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    if yawning_counter >= yawning_thresh:
        last_detected_yawn = datetime.now()
        cv2.putText(frame, 'ALERT: Yawning', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    elif (datetime.now() - last_detected_yawn).total_seconds() < 2:
        cv2.putText(frame, 'ALERT: Yawning', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    # new_frame_time = time.time()
    
    # fps = 1/(new_frame_time-prev_frame_time)
    # prev_frame_time = new_frame_time
    
    fps = int(fps)
    fps = str(fps)

    # cv2.putText(frame, fps, (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2, cv2.LINE_AA)

    cv2.imshow('YOLO', results[0].plot())
    latency.append(results[0].speed)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 microsleep, 45.1ms
Speed: 3.0ms preprocess, 45.1ms inference, 80.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 16.5ms
Speed: 2.0ms preprocess, 16.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 neutral, 14.1ms
Speed: 2.0ms preprocess, 14.1ms inference, 2.0ms postprocess per image at sha

KeyboardInterrupt: 